# Syracuse Real Estate Investment Trust: Initial Analysis

The Syracuse Real Estate Investment Trust (SREIT) would like to select a geographic area to focus their investment activities. The primary goal is to find an area with higher-than-average return on investment (ROI). The Zillow Home Value Index contains that monthly median single family residence price for 30,464 zip codes.

Data Source: Zillow Home Value Index (ZHVI): [files.zillowstatic.com/research/public/Zip/Zip_Zhvi_SingleFamilyResidence.csv](files.zillowstatic.com/research/public/Zip/Zip_Zhvi_SingleFamilyResidence.csv)

One of the challenges with the ZHVI data is that not all the zip codes have complete data for the 24-year period between 1996 and 2020. In an effort to preserve all of the observations, a subset was created to compare the growth rate of property values over a five-year period, 2015-2020. 

The mean five-year ROI for single family residences was 26.2% with a 16.6% standard deviation. The median ROI was 24.5%. The upper quartile for ROI is between 33.9% and a dizzying 248%. The upper quartile was sorted to rank the top 30 zip codes in terms of ROI.

There are several geographic areas of note including Columbus-Ohio, Tampa-Florida, Beverly Hills-California, and Tacoma-Washington. The most significant metro areas are ones with several high growth zip codes in the top 30. Atlanta-Sandy Springs-Roswell in Georgia has six zip codes in the top 30. Dallas-Fort Worth-Arlington in Texas has ten.

In [1]:
# Load Packages

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from fbprophet import Prophet

import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Read in and inspect the Zip Code Data file
data = pd.read_csv('Zip_Zhvi_SingleFamilyResidence.csv')

# Select relavant attributes
dataID = data[['RegionName', 'City', 'Metro']]
dataX = data.iloc[:, 240:300]

# Concatonate the columns to one data frame
dataSubset = pd.concat([dataID, dataX.reindex(dataID.index)], axis=1)

# Fill Missing City data and drop NA for the SFR values
dataSubset.City.fillna(value = "Unidentified")
dataSubset.dropna(inplace=True)

# Add a column for 5 year ROI
dataSubset['growth5yr'] = ((dataSubset['3/31/2020'] - dataSubset['4/30/2015']) / dataSubset['4/30/2015'])

print(dataSubset.head())
print('Data Shape:', dataSubset.shape)

   RegionName      City                             Metro  4/30/2015  \
0       10025  New York       New York-Newark-Jersey City  1356525.0   
1       60657   Chicago          Chicago-Naperville-Elgin   903862.0   
2       10023  New York       New York-Newark-Jersey City  1495385.0   
3       77494      Katy  Houston-The Woodlands-Sugar Land   334607.0   
4       60614   Chicago          Chicago-Naperville-Elgin  1098788.0   

   5/31/2015  6/30/2015  7/31/2015  8/31/2015  9/30/2015  10/31/2015  ...  \
0  1350097.0  1355930.0  1362446.0  1375476.0  1380169.0   1388270.0  ...   
1   912966.0   915000.0   916602.0   920152.0   923585.0    923550.0  ...   
2  1494490.0  1506862.0  1513489.0  1515314.0  1517631.0   1517245.0  ...   
3   336254.0   337852.0   338657.0   339058.0   339256.0    339524.0  ...   
4  1110803.0  1114983.0  1119110.0  1123493.0  1126324.0   1126873.0  ...   

   7/31/2019  8/31/2019  9/30/2019  10/31/2019  11/30/2019  12/31/2019  \
0    1405862    1402547    139

In [3]:
# Display descriptive statistics for the dataset
dataSubset.growth5yr.describe()

count    22799.000000
mean         0.261567
std          0.166180
min         -0.387390
25%          0.157901
50%          0.245272
75%          0.338850
max          2.480128
Name: growth5yr, dtype: float64

In [4]:
# Create a subset of the top 25% of zip codes by 5-year ROI 
dataSubset1 = dataSubset[(dataSubset["growth5yr"] > 0.33)]

# Sort by 5-year ROI
dataSubset1.sort_values(by=['growth5yr'], inplace=True, ascending=False)

# Create a dataset for the top 30 zip codes
dataTop30 = dataSubset1.iloc[0:30,:]
dataTop30.reset_index(drop=True, inplace=True)

# Write the table to a csv
dataTop30.to_csv(r'dataTop30.csv', index = False)

dataTop30

c:\users\nosta\.conda\envs\v-env\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,RegionName,City,Metro,4/30/2015,5/31/2015,6/30/2015,7/31/2015,8/31/2015,9/30/2015,10/31/2015,...,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,growth5yr
0,43205,Columbus,Columbus,59732.0,59449.0,59635.0,60328.0,61688.0,62798.0,63811.0,...,190884,194402,197836,200347,202565,204400,205186,206203,207875,2.480128
1,30314,Atlanta,Atlanta-Sandy Springs-Roswell,54298.0,55001.0,55316.0,56172.0,56842.0,57285.0,57394.0,...,157172,159175,161203,163443,166123,169741,174383,180050,184547,2.398781
2,30310,Atlanta,Atlanta-Sandy Springs-Roswell,78299.0,79488.0,80291.0,80787.0,81228.0,82167.0,83847.0,...,228698,229073,229733,230116,231321,232872,235007,237397,238612,2.047446
3,32208,Jacksonville,Jacksonville,39348.0,38848.0,38454.0,38537.0,39057.0,39365.0,39521.0,...,100501,101266,101774,102537,102687,103488,104364,105567,107090,1.721612
4,30311,Atlanta,Atlanta-Sandy Springs-Roswell,81877.0,83881.0,84995.0,85852.0,85729.0,86212.0,86945.0,...,195887,196764,197597,199317,201088,203973,206085,208123,208673,1.548616
5,30032,Candler-Mcafee,Atlanta-Sandy Springs-Roswell,83564.0,83412.0,82708.0,82401.0,82830.0,83877.0,84646.0,...,204159,205479,206550,207828,208568,209173,209845,210278,211567,1.531796
6,75224,Dallas,Dallas-Fort Worth-Arlington,73182.0,74731.0,76706.0,78521.0,79636.0,80401.0,81311.0,...,171017,172292,174196,176359,178328,179770,181452,183072,184503,1.521153
7,90211,Beverly Hills,Los Angeles-Long Beach-Anaheim,1010992.0,1029976.0,1047114.0,1065646.0,1079371.0,1097075.0,1118852.0,...,2253956,2276149,2300187,2329400,2354435,2384830,2423355,2485106,2544209,1.516547
8,75141,Hutchins,Dallas-Fort Worth-Arlington,65704.0,66877.0,68252.0,69665.0,70089.0,70460.0,71091.0,...,156644,157846,159603,160230,160677,160674,161414,162083,162905,1.479377
9,33605,Tampa,Tampa-St. Petersburg-Clearwater,61869.0,62487.0,63497.0,64472.0,65326.0,66166.0,67646.0,...,144583,145637,146810,148029,149508,150954,152098,152722,153161,1.475569
